In [3]:
# Import all dependencies

#--- URL and File related libraries
import requests
import os

#--- Data extraction, transformation & loading libraries
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
import pymongo

#--- Utilities libraries
from datetime import datetime as dt

#--- api key file
from config1 import OMDB_api_key

### Set Global variables accesible for notebook scope

In [4]:
# Declare TMDB file to load
file = "data/tmdb_5000_movies.csv"

# Set the OMDB api url to extract JSON data
url = f"http://www.omdbapi.com/?apikey={OMDB_api_key}&t="

### Extract TMDB data from csv to dataframe

In [5]:
df = pd.read_csv(file, low_memory = False)

#print top 3 records
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


#### Describe and gather dataframe information

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
budget                  4803 non-null int64
genres                  4803 non-null object
homepage                1712 non-null object
id                      4803 non-null int64
keywords                4803 non-null object
original_language       4803 non-null object
original_title          4803 non-null object
overview                4800 non-null object
popularity              4803 non-null float64
production_companies    4803 non-null object
production_countries    4803 non-null object
release_date            4802 non-null object
revenue                 4803 non-null int64
runtime                 4801 non-null float64
spoken_languages        4803 non-null object
status                  4803 non-null object
tagline                 3959 non-null object
title                   4803 non-null object
vote_average            4803 non-null float64
vote_count              4803 non-null 

In [4]:
df.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [9]:
df_filtered = df[df.release_date > "1999-31-12"].sort_values(by='release_date', ascending=True)

### OMDB extraction

In [11]:
movies_df = df_filtered[['title','release_date']]
movies_df.count()

title           3494
release_date    3494
dtype: int64

In [29]:
# check to see if any duplicates exists
print(f"Number of movies with duplicate title : {movies_df['title'].duplicated().sum()}")

# get the movies that are duplicated ad check to see if they are re-makes
movies_df.loc[movies_df['title'] == movies_df[movies_df['title'].duplicated()].title.values[0],]

Number of movies with duplicate title : 1


,title,release_date
2877,The Host,2006-07-27
972,The Host,2013-03-22


In [64]:
# Create a function to connect to OMDB to read the data
def getOMDB_movies(url, movies_list):
    responses = [];
    
    for movie in movies_list:
        movie_data = requests.get(url + movie).json()
        responses.append(movie_data)
    
    return responses

In [ ]:
title_movies = movies_df['title'].tolist()

#call the function to get OMDB data -- OPEN IT WHEN NEEDED TO READ FROM API

#responses = getOMDB_movies(url,title_movies)

#omdb_responses_df = json_normalize(responses)
#omdb_responses_df.head()

In [70]:
# for testing purposes so we do not exhaust the key, read from csv
omdb_file = "omdb_api.csv"
omdb_responses_df = pd.read_csv(omdb_file)

#### ++++++++++++++++++++ Extraction Complete +++++++++++++++++++++++++

#### ++++++++++++++++++++ Transformation Begins +++++++++++++++++++++++++

#### TMDB transformation

In the transformation, we will perform 

- Data Filtering
- Data Cleaning
- Data Changes like addition of new columns, change formating etc.,
- Data Grouping and Aggregations, if needed

### Data Filtering (specific colums, row conditions)

In [33]:
# extract only required cols
tmdb_df = df_filtered[['title','budget','popularity','production_companies','release_date','runtime','revenue',
                          'status','vote_average','vote_count']]
tmdb_df.head()

,title,budget,popularity,production_companies,release_date,runtime,revenue,status,vote_average,vote_count
2772,The Widow of Saint-Pierre,0,1.780065,"[{""name"": ""Cin\u00e9maginaire Inc."", ""id"": 280...",2000-01-01,112.0,0,Released,6.7,11
3141,Next Friday,11000000,9.337388,"[{""name"": ""New Line Cinema"", ""id"": 12}]",2000-01-12,98.0,59827328,Released,6.4,135
3344,My Dog Skip,7000000,5.675535,"[{""name"": ""Alcon Entertainment"", ""id"": 1088}, ...",2000-01-14,95.0,0,Released,6.5,69
775,Supernova,90000000,5.762037,"[{""name"": ""United Artists"", ""id"": 60}, {""name""...",2000-01-14,91.0,14828081,Released,4.9,109
4636,Chuck & Buck,0,0.812855,[],2000-01-21,96.0,0,Released,5.7,16


### TMDB Data Cleaning

Tasks performed will be,
 - filter out columns not needed
 - remove rows with NA in Title and Release_Date
 - fill na with mean (if numeric), "not provided" (if column is categorical)
 - Check if title is duplicated, if it is check if the release date is different to make sure it is a remake.
     - else drop it


In [36]:
# Check to see if any null values exist in key colums - title and release date
print(f"Missing values in release date : {tmdb_df.release_date.isnull().sum()}")

print(f"Missing values in title : {tmdb_df.title.isnull().sum()}")
#tmdb_df.count()

Missing values in release date : 0
Missing values in title : 0


#### Drop row with null values (safe method) 

In [39]:
tmdb_df = tmdb_df.dropna(axis = 0, subset = ['title','release_date'])
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3494 entries, 2772 to 4255
Data columns (total 10 columns):
title                   3494 non-null object
budget                  3494 non-null int64
popularity              3494 non-null float64
production_companies    3494 non-null object
release_date            3494 non-null object
runtime                 3492 non-null float64
revenue                 3494 non-null int64
status                  3494 non-null object
vote_average            3494 non-null float64
vote_count              3494 non-null int64
dtypes: float64(3), int64(3), object(4)
memory usage: 300.3+ KB


#### Based on above info, we see that runtime has NA value, fill it with mean runtime

In [43]:
runtime_mean = round(df.runtime.mean(),2)

tmdb_df[['runtime']] = tmdb_df[['runtime']].fillna(value = runtime_mean)
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3494 entries, 2772 to 4255
Data columns (total 10 columns):
title                   3494 non-null object
budget                  3494 non-null int64
popularity              3494 non-null float64
production_companies    3494 non-null object
release_date            3494 non-null object
runtime                 3494 non-null float64
revenue                 3494 non-null int64
status                  3494 non-null object
vote_average            3494 non-null float64
vote_count              3494 non-null int64
dtypes: float64(3), int64(3), object(4)
memory usage: 300.3+ KB


In [45]:
# check to see if all null-values are taken care of
tmdb_df.isnull().sum()

title                   0
budget                  0
popularity              0
production_companies    0
release_date            0
runtime                 0
revenue                 0
status                  0
vote_average            0
vote_count              0
dtype: int64

In [63]:
if(tmdb_df.title.duplicated().sum() > 0):
    # then title is duplicated
    # check if it is a remake by cmpaoring release dat
    if(tmdb_df[['title','release_date']].duplicated().sum() > 0):
        # titles are duplicated, save it in duplicate df 
        duplicate_movies_tmdb = tmdb_df[tmdb_df[['title','release_date']].duplicated()]
        # drop duplicated movie, keep only the first one
        tmdb_df.drop_duplicates(subset = ['title','release_date'], keep = 'first', inplace = True)
    else:
        print("No duplicates, only remakes!")

No duplicates, only remakes!


### Data Modifications
Tasks will be 
 - Adding a new column called 'Year' used for joining with OMDB

In [60]:
# create Year column
tmdb_df['Year'] = tmdb_df['release_date'].apply(lambda r: r.split("-")[0])
tmdb_df.Year.head()


2772    2000
3141    2000
3344    2000
775     2000
4636    2000
Name: Year, dtype: object

### Save extraction and transformation output to csv

In [62]:
# write it to CSV
tmdb_df.to_csv("TMDB_Cleaned.csv")

### OMDB transformation

In the transformation, we will perform 

- Data Filtering
- Data Cleaning
- Data Changes like addition of new columns, change formating etc.,
- Data Grouping and Aggregations, if needed

### OMDB Data Filtering

In [71]:
# filter only the required fields
omdb_df = omdb_responses_df[['Title','Year','Genre', 'Awards', 'Metascore', 'BoxOffice']]

omdb_df = omdb_df.sort_values(by='Year', ascending=True)

omdb_df.count()

Title        3362
Year         3362
Genre        3362
Awards       2877
Metascore    2987
BoxOffice    2437
dtype: int64

In [72]:
omdb_df.head()

,Title,Year,Genre,Awards,Metascore,BoxOffice
0,The Widow of Saint-Pierre,2000,"Drama, History, Romance",Nominated for 1 Golden Globe. Another 3 wins &...,73.0,NaN
105,Supernova,2000,"Horror, Sci-Fi, Thriller",NaN,19.0,NaN
106,Next Friday,2000,Comedy,1 nomination.,41.0,NaN
107,Thomas and the Magic Railroad,2000,"Adventure, Comedy, Drama, Family, Fantasy",2 nominations.,19.0,"$2,027,042"
108,Down to You,2000,"Comedy, Drama, Romance",1 win & 3 nominations.,13.0,NaN


### OMDB Data Cleaning
Tasks performed will be,
 - remove rows with NA in Title and Year
 - fill na with mean (if numeric), "not provided" (if column is categorical)
 - Check if title is duplicated, if it is check if the release date is different to make sure it is a remake.
     - else drop it

In [75]:
# Drop any rows where Title and Year are NA
new_movies = omdb_df.dropna(axis = 0 , subset = ['Title','Year'])
new_movies.count()

Title        3362
Year         3362
Genre        3362
Awards       2877
Metascore    2987
BoxOffice    2437
dtype: int64

In [76]:
# Awards has NAN values, fill it with value "None"
new_movies[['Awards']] = new_movies[['Awards']].fillna(value = 'none')
new_movies.count()

In [80]:
omdb_movies_withNoMetaScore = new_movies[new_movies.Metascore.isnull()]

# where no metascore is unavailable, make it 0
new_movies[['Metascore']] = new_movies[['Metascore']].fillna(value = 0)

new_movies.count()

Title        3362
Year         3362
Genre        3362
Awards       3362
Metascore    3362
BoxOffice    2437
dtype: int64

In [82]:
omdb_movies_withNA_boxoff = new_movies[new_movies.BoxOffice.isnull()]

# where no Box Office revenue is provied, make it 0
new_movies[['BoxOffice']] = new_movies[['BoxOffice']].fillna(value = '$0')

new_movies.count()

Title        3362
Year         3362
Genre        3362
Awards       3362
Metascore    3362
BoxOffice    3362
dtype: int64

In [85]:
new_movies[["Year"]] = new_movies.Year.apply(lambda y: int(y))

# Filter movies only from 1999
cool_movies = new_movies[new_movies.Year > 1999].sort_values(by='Year', ascending=True)

cool_movies.count()

Title        3362
Year         3362
Genre        3362
Awards       3362
Metascore    3362
BoxOffice    3362
dtype: int64

In [88]:
### CHeck to see if any null exists
cool_movies.isnull().sum()

Title        0
Year         0
Genre        0
Awards       0
Metascore    0
BoxOffice    0
dtype: int64

In [91]:
# check if movies are duplicated
if(cool_movies.Title.duplicated().sum() > 0):
    # then title is duplicated
    # check if it is a remake by cmpaoring release dat
    if(cool_movies[['Title','Year']].duplicated().sum() > 0):
        # titles are duplicated, save it in duplicate df 
        duplicate_movies_omdb = cool_movies[cool_movies[['Title','Year']].duplicated()]
        # drop duplicated movie, keep only the first one
        cool_movies.drop_duplicates(subset = ['Title','Year'], keep = 'first', inplace = True)
    else:
        print("No duplicates, only remakes!")
else:
        print("No duplicates, only remakes!")



No duplicates, only remakes!


### Save the cleaned OMDB file to local drive

In [92]:
cool_movies.to_csv('omdb_api_cleaned.csv')

#### ++++++++++++++++++++++++++++++ Transformation - Done +++++++++++++++++++++++++++++++++

- check the total records inserted### Data Exploration - Optional

In [47]:
movies_noBudget = tmdb_df[tmdb_df.budget <= 0 ]
print(f"Movies with Zero budget : {movies_noBudget.title.count()}")

Movies with Zero budget : 798


In [46]:
movies_noRevenue = tmdb_df[tmdb_df.revenue <= 0 ]
print(f"Movies that have not produced any revenue : {movies_noRevenue.title.count()}")

Movies that have not produced any revenue : 1093


In [48]:
movies_withNoVotes = tmdb_df[tmdb_df.vote_count <= 0 ]
print(f"Movies with zero votes : {movies_withNoVotes.title.count()}")

Movies with zero votes : 56


In [50]:
print(f"Status categories are : {tmdb_df.status.unique()}")

Status categories are : ['Released' 'Rumored' 'Post Production']
